In [ ]:
from datetime import date
from IPython.display import display, Markdown
import pandas
import pathlib

In [ ]:
pandas.set_option("display.max_columns", None)
pandas.set_option("display.max_rows", None)

In [ ]:
OUTPUT_DIR = pathlib.Path("../output")

In [ ]:
rows = pandas.read_csv(OUTPUT_DIR / "rows.csv")

# OpenSAFELY-TPP Database Schema

This report displays the schema of the OpenSAFELY-TPP database.
It is part of the OpenSAFELY platform's technical documentation
and is published at <https://reports.opensafely.org/>.

The schema is read from the `OpenSAFELYSchemaInformation` table,
which is refreshed at the same time as the core SystmOne primary care datasets.
There are some non-automated steps required to update the schema information when a new table is added to the database &mdash;
if you can't see a table that you are expecting to see, then please speak to TPP.

If you would like to use the OpenSAFELY platform, then you should read
our [documentation](https://docs.opensafely.org/),
our [principles](https://www.opensafely.org/about/),
and our process for [onboarding new users](https://www.opensafely.org/onboarding-new-users/).
If you would like to see the code we used to create this report,
then you can [view it on GitHub](https://github.com/opensafely/tpp-database-schema).

## Data Sources

The core SystmOne primary care datasets are stored in the `S1` tables in the database.
The table below lists other, externally-linked datasets:

| Dataset                                                  | Source                           | Tables                                             |
| -------------------------------------------------------- | -------------------------------- | -------------------------------------------------- |
| All positive or negative SARS-CoV2 tests                 | SGSS                             | `SGSS_AllTests_Positive`, `SGSS_AllTests_Negative` |
| First-ever positive or negative SARS-CoV2 test           | SGSS                             | `SGSS_Positive`, `SGSS_Negative`                   |
| A&E attendances                                          | SUS Emergency Care               | `EC`                                               |
| In-patient hospital admissions                           | SUS Admitted Patient Care Spells | `APCS`                                             |
| Out-patient hospital appointments                        | SUS                              | `OPA`                                              |
| Covid-related ICU admissions                             | ICNARC                           | `ICNARC`                                           |
| Covid-related in-hospital deaths                         | CPNS                             | `CPNS`                                             |
| COVID-19 Infection Survey                                | ONS                              | `ONS_CIS`                                          |
| All-cause registered deaths                              | ONS                              | `ONS_Deaths`                                       |
| High cost drugs                                          |                                  | `HighCostDrugs`                                    |
| Unique Property Reference Number<sup>1</sup>             |                                  | `UPRN`                                             |
| Master Patient Index                                     |                                  | `MPI`                                              |
| Health and Social Care Worker identification<sup>2</sup> |                                  | `HealthCareWorker`                                 |

<sup>1</sup>Used for deriving household variables<br>
<sup>2</sup>Collected at the point of vaccination

Some of these tables are accompanied by additional tables.
For example, the `OPA` table contains the core out-patient appointment event data, and is accompanied by the `OPA_Cost`, `OPA_Diag`, and `OPA_Proc` tables.

## Report Run Date

In [ ]:
Markdown(
    f"""
This report was run on {date.today():%Y-%m-%d}.
It reflects the state of the database on this date.
"""
)

## Table Names by Data Source

In [ ]:
rows[["DataSource", "TableName"]].drop_duplicates()

## Table Schemas

The table schemas contain the following information:

* `ColumnName`, the column name
* `ColumnType`, the column type, such as integer, numeric, or date. For more information, see [SQL Server's _data types_ documentation][1]
* `Precision`, `Scale` and `MaxLength`. For more information, see [SQL Server's _precision, scale, and length_ documentation][2]
* `IsNullable`, whether `NULL` values are accepted

[1]: https://docs.microsoft.com/en-us/sql/t-sql/data-types/data-types-transact-sql
[2]: https://docs.microsoft.com/en-us/sql/t-sql/data-types/precision-scale-and-length-transact-sql

Use the links in the list below to jump to a table schema.

In [ ]:
toc = []
sections = []
for data_source, by_data_source in rows.groupby("DataSource"):
    del by_data_source["DataSource"]
    toc.append(f"* [{data_source}](#{data_source})")
    sections.append(f"### {data_source}")

    for table_name, by_table_name in by_data_source.groupby("TableName"):
        del by_table_name["TableName"]
        toc.append(f"  * [{table_name}](#{table_name})")
        sections.append(f"### {table_name}")
        sections.append(by_table_name.set_index("ColumnName").to_html())

In [ ]:
Markdown("\n".join(toc))

In [ ]:
Markdown("\n\n".join(sections))